# **NAME: PARTHA SAKHA PAUL**

# **ROLL: MA23M016**

    CS6910_assignment 2


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np

# data_dir = '/content/drive/My Drive/nature_12K'


# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# data_dir = '/content/drive/My Drive/nature_12K/inaturalist_12K'

In [2]:
def prepare_data_loaders(data_augment, batch_size=64):
    if data_augment:
        # With data augmentation
        train_transform = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.450, 0.470, 0.490], std=[0.200, 0.220, 0.240]),
        ])
#         val_transform = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.450, 0.470, 0.490], std=[0.200, 0.220, 0.240]),
#         ])
    else:
        # Without data augmentation
        train_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        ])
#         val_transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         ])
    
    # Load datasets
    train_data = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/train', transform=train_transform)
#     validation_dataset = datasets.ImageFolder(root='/kaggle/input/inaturalist/inaturalist_12K/val', transform=val_transform)
#     print(len(train_data))
    # print(train_data[9998])
    # Creating data indices for training and validation splits:
    dataset_size = len(train_data)  #9999
    indices = list(range(dataset_size)) #[0,1,...,9998]
    split = int(np.floor(0.2 * dataset_size))  #1999
#     print(split)
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
    # print((val_indices))

    # Creating data samplers and loaders
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)
    # print(type(train_sampler))

    train_loader = DataLoader(train_data, batch_size=64, sampler=train_sampler)
    # print(len(train_loader))
    validation_loader = DataLoader(train_data, batch_size=64, sampler=valid_sampler)
    # print(*validation_loader)
    # Prepare data loaders
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, validation_loader

In [3]:
activation_functions = {
            'relu': F.relu,
            'gelu': F.gelu,
            'silu': F.silu,
            'mish': F.mish
        }

In [3]:
# # Defining transforms for the training, validation, and testing sets
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# data_dir = '/kaggle/input/inaturalist/inaturalist_12K/train'

# # Load the datasets with ImageFolder
# train_data = datasets.ImageFolder(data_dir, transform=transform)
# print(len(train_data))
# # print(train_data[9998])
# # Creating data indices for training and validation splits:
# dataset_size = len(train_data)  #9999
# indices = list(range(dataset_size)) #[0,1,...,9998]
# split = int(np.floor(0.2 * dataset_size))  #1999
# print(split)
# np.random.shuffle(indices)
# train_indices, val_indices = indices[split:], indices[:split]  # [1999,...,9998] , [0,...,1998]
# # print((val_indices))

# # Creating data samplers and loaders
# train_sampler = SubsetRandomSampler(train_indices)
# valid_sampler = SubsetRandomSampler(val_indices)
# # print(type(train_sampler))

# train_loader = DataLoader(train_data, batch_size=64, sampler=train_sampler)
# # print(len(train_loader))
# validation_loader = DataLoader(train_data, batch_size=64, sampler=valid_sampler)
# # print(*validation_loader)

9999
1999


In [10]:
# defining a class CNN that inherits from nn.Module which is PyTorch's base class for all neural network
class my_CNN(nn.Module):
    def __init__(self, num_classes=10, num_filters=[32, 64, 128, 256, 512], filter_sizes=[3, 3, 3, 3, 3], activation_fn='relu', num_neurons_dense = 512, dropout_rate = 0.5, use_batchnorm='no'):
        super(my_CNN, self).__init__()
        # Initializing class variables
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.activation_fn = activation_functions[activation_fn]
        self.num_neurons_dense = num_neurons_dense
        self.dropout_rate = dropout_rate
        # Convert yes/no to boolean
        self.use_batchnorm = True if use_batchnorm.lower() == 'yes' else False

        # Creating convolution layers using ModuleList
        self.conv_layers = nn.ModuleList()
        # List to hold batchnorm layers for conv
        self.batchnorm_layers_conv = nn.ModuleList()
        in_channels = 3  # since, input images are RGB
        for out_channels, kernel_size in zip(num_filters, filter_sizes):
            # a convolution layer with in-out channels and kernel size
            conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
            self.conv_layers.append(conv_layer)
            if self.use_batchnorm:
                # Add a Batchnorm2d layer for each conv layer
                self.batchnorm_layers_conv.append(nn.BatchNorm2d(out_channels))
            # Update in_channels for the next layer
            in_channels = out_channels

            
        # Defining dropout before it's used
        self.dropout = nn.Dropout(dropout_rate)
        # find the size needed to flatten the conv layer outputs, initializing the dense layer accordingly
        self.init_flatten_size(input_shape=(3, 224, 224))  # input images are 224x224 RGB images
        # initializing the dense layer from flatten size
        self.dense = nn.Linear(self.flatten_size, num_neurons_dense)
        if self.use_batchnorm:
             # Batchnorm1d for the dense layer
            self.batchnorm_dense = nn.BatchNorm1d(num_neurons_dense)
        # output layer: maps from the dense layer to the number of classes.
        self.out = nn.Linear(num_neurons_dense, num_classes)


    def init_flatten_size(self, input_shape):
        # to disable gradient calculations for speed up this computation
        with torch.no_grad():
            # a dummy input tensor of the correct shape
            input_tensor = torch.zeros(1, *input_shape)
            # Forward prop through the conv layers to find output size
            output = self.forward_conv_layers(input_tensor)
            # total number of output features is the flat size needed for the dense layer input
            self.flatten_size = output.numel()

    # a function for sequentially passing input through all convolutional layers and applying activation and pooling
    def forward_conv_layers(self, x):
        # Convolution layer
        for i, conv in enumerate(self.conv_layers):
            x = conv(x)  # Convolution operation
            if self.use_batchnorm:
                # Apply batchnorm if enabled
                x = self.batchnorm_layers_conv[i](x)
            x = self.activation_fn(x)  # Apply the specified activation function
            x = F.max_pool2d(x, 2)  # max pooling with a kernel size of 2
            x = self.dropout(x)  # Apply dropout after pooling
        return x

    # forward method defines how the input tensor is transformed through the model
    def forward(self, x):
        # Passing input through the convolution blocks
        x = self.forward_conv_layers(x)
        # Flattening the output for the dense layer
        x = torch.flatten(x, 1)   # Flatten the output for the dense layer
        # Dense layer
        x = self.dense(x)
        if self.use_batchnorm:
            # Apply batchnorm if enabled to dense layer
            x = self.batchnorm_dense(x)
        x = self.activation_fn(x)
        # Applying dropout before the final layer
        x = self.dropout(x)
        # Output layer
        x = self.out(x)
        return x
    

    # Method for training and evaluating the model
    def train_and_evaluate(self, train_loader, validation_loader, n_epochs=10, lr=0.001, device=None):
        if device is None:
            # setting up CUDA if available, otherwise, using CPU
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)  # transferring the model to the selected device (GPU or CPU)

        # Initializing the loss function and optimizer
        loss_function = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)  # self.parameters() is a built-in method provided by PyTorch's nn.Module that collects all the learnable parameters of our model

        # Training loop
        for epoch in range(n_epochs):
            self.train()  # Setting model to training mode
            training_loss = 0.0
            # tracking number of correctly predicted instances
            correct_train = 0
            # tracking total number of instances
            total_train = 0

            # Iterate over the training data
            for images, labels in train_loader:
                # transferring images and labels to the current device (GPU or CPU)
                images, labels = images.to(device), labels.to(device)
                # clearing the gradients of all optimized variables
                optimizer.zero_grad()
                # Forward prop: computing predicted outputs by passing inputs to the model
                outputs = self(images)
                # calculating the loss between predicted and true labels
                loss = loss_function(outputs, labels)
                # Backward prop: computing gradient of the loss with respect to parameters
                loss.backward()
                # optimization step (parameter update)
                optimizer.step()
                # updating running training loss
                training_loss += loss.item() * images.size(0)
                # converting output probabilities to predicted class
                _, predicted_train = torch.max(outputs.data, 1)
                # updating total number of instances
                total_train += labels.size(0)
                # updating correctly predicted instances
                correct_train += (predicted_train == labels).sum().item()

            # average loss over an epoch
            training_loss /= len(train_loader.sampler)
            # train accuracy
            train_accuracy = correct_train / total_train

            # Evaluation phase
            self.eval()  # Setting model to evaluation mode
            # initializing the validation loss for the epoch
            validation_loss = 0.0
            # tracking number of correctly predicted instances
            correct_val = 0
            # tracking total number of instances
            total_val = 0
            # disable gradient calculation for validation, saving memory and computations
            with torch.no_grad():
                # iterating over the validation data loader
                for images, labels in validation_loader:
                    # transferring images and labels to the current device (GPU or CPU)
                    images, labels = images.to(device), labels.to(device)
                    # Forward prop: computing predicted outputs by passing inputs to the model
                    outputs = self(images)
                    # calculating the loss between predicted and true labels
                    loss = loss_function(outputs, labels)
                    # updating running validation loss
                    validation_loss += loss.item() * images.size(0)
                    # converting output probabilities to predicted class
                    _, predicted = torch.max(outputs.data, 1)
                    # updating total number of instances
                    total_val += labels.size(0)
                    # updating correctly predicted instances
                    correct_val += (predicted == labels).sum().item()

            # average validation loss over an epoch
            validation_loss /= len(validation_loader.sampler)
            # validation accuracy
            val_accuracy = correct_val / total_val

            # printing training and validation results
            print(f'Epoch {epoch+1}, Training Loss: {training_loss:.4f}, Training Accuracy: {train_accuracy*100:.2f}, \nValidation Loss: {validation_loss:.4f}, Validation Accuracy: {val_accuracy*100:.2f}')
            wandb.log({"train_accuracy": train_accuracy * 100, "training_loss": training_loss})
            wandb.log({"val_accuracy": val_accuracy * 100, "val_loss": validation_loss})


# model with parameters
model = my_CNN(num_classes=10,
                  num_filters=[32, 64, 128, 256, 512],
                  filter_sizes=[7,9,7,9,9],
                  activation_fn='relu',
                  num_neurons_dense=512,
                  dropout_rate = 0.5, 
                  use_batchnorm='yes')

print(model)

# model.train_and_evaluate(train_loader, validation_loader, n_epochs=10, lr=0.001)

my_CNN(
  (conv_layers): ModuleList(
    (0): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
    (3): Conv2d(128, 256, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
    (4): Conv2d(256, 512, kernel_size=(9, 9), stride=(1, 1), padding=(1, 1))
  )
  (batchnorm_layers_conv): ModuleList(
    (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (dense): Linear(in_features=512, out_feature

In [5]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

key = input('Enter your API:')
wandb.login(key=key)  #25c2257eaf6c22aa056893db14da4ee2bf0a531a

Enter your API: 25c2257eaf6c22aa056893db14da4ee2bf0a531a


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
sweep_config = {
    'method': 'bayes',  # can be grid, random
    'name' : 'cnn kaggle all 2',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'num_filters': {
            'values': [[16, 32, 64, 128, 256], [64, 64, 64, 64, 64], [256, 128, 64, 32, 16]] 
        },
        'filter_sizes':{
            'values': [[3,3,3,3,3], [3,3,5,3,3], [3,5,3,5,3]]
        },
        'activation_fn': {
            'values': ['relu', 'gelu', 'silu', 'mish']
        },
        'dropout': {
            'values': [0.2, 0.3, 0.5]
        },
        'batch_norm':{x
            'values': ['true','false']
        },
        'data_augment': {
            'values': ['true', 'false']
        },
        'num_neurons_dense':{
            'values': [64, 128, 256]
        }
    }
}

sweep_id = wandb.sweep(sweep = sweep_config, project="Deep_learning_A2")

Create sweep with ID: hso5osa8
Sweep URL: https://wandb.ai/parthasakhapaul/Deep_learning_A2/sweeps/hso5osa8


In [ ]:
def main():
    # Initialize a new wandb run
    with wandb.init() as run:
        run_name = "-ac-"+wandb.config.activation_fn+"-filters-"+str(wandb.config.num_filters)+"-filt_size-"+str(wandb.config.filter_sizes)+"-dropout-"+str(wandb.config.dropout)+"-batch_norm-"+wandb.config.batch_norm+"-data_aug-"+wandb.config.data_augment+"-num_neurons_dense-"+str(wandb.config.num_neurons_dense)
        wandb.run.name=run_name
        # Model object creation
        model = my_CNN(num_classes = 10,
                  num_filters = wandb.config.num_filters,
                  filter_sizes = wandb.config.filter_sizes,
                  activation_fn = wandb.config.activation_fn,
                  num_neurons_dense = wandb.config.num_neurons_dense,
                  dropout_rate = wandb.config.dropout,
                  use_batchnorm = wandb.config.batch_norm)
        
        data_augment = wandb.config.data_augment
        train_loader, validation_loader = prepare_data_loaders(data_augment)
        # Model training
        model.train_and_evaluate(train_loader, validation_loader, n_epochs=10, lr=0.001)        

wandb.agent(sweep_id, function=main, count=10)  # to run 5 experiments
wandb.finish()

wandb: Agent Starting Run: ncgek4sw with config:
wandb: 	activation_fn: gelu
wandb: 	batch_norm: false
wandb: 	data_augment: false
wandb: 	dropout: 0.3
wandb: 	filter_sizes: [3, 5, 3, 5, 3]
wandb: 	num_filters: [16, 32, 64, 128, 256]
wandb: 	num_neurons_dense: 64


Epoch 1, Training Loss: 2.2494, Training Accuracy: 16.51, 
Validation Loss: 2.2277, Validation Accuracy: 17.56
Epoch 2, Training Loss: 2.1907, Training Accuracy: 19.48, 
Validation Loss: 2.1591, Validation Accuracy: 19.01
Epoch 3, Training Loss: 2.1522, Training Accuracy: 21.14, 
Validation Loss: 2.1952, Validation Accuracy: 19.66
Epoch 4, Training Loss: 2.1171, Training Accuracy: 22.69, 
Validation Loss: 2.1397, Validation Accuracy: 20.66
Epoch 5, Training Loss: 2.1050, Training Accuracy: 24.11, 
Validation Loss: 2.1016, Validation Accuracy: 24.26
Epoch 6, Training Loss: 2.0828, Training Accuracy: 25.25, 
Validation Loss: 2.1299, Validation Accuracy: 24.51
Epoch 7, Training Loss: 2.0764, Training Accuracy: 25.14, 
Validation Loss: 2.0650, Validation Accuracy: 25.71
Epoch 8, Training Loss: 2.0565, Training Accuracy: 25.87, 
Validation Loss: 2.0664, Validation Accuracy: 24.71
Epoch 9, Training Loss: 2.0338, Training Accuracy: 26.70, 
Validation Loss: 2.0946, Validation Accuracy: 23.91
E

train_accuracy,▁▃▄▅▆▇▇▇██
training_loss,█▆▅▄▄▃▃▂▁▁
val_accuracy,▁▂▃▄▇▇█▇▆█
val_loss,█▅▇▄▃▄▁▁▂▂
train_accuracy,27.1375
training_loss,2.0237
val_accuracy,25.31266
val_loss,2.08188


wandb: Agent Starting Run: nzjoqa3a with config:
wandb: 	activation_fn: silu
wandb: 	batch_norm: true
wandb: 	data_augment: true
wandb: 	dropout: 0.3
wandb: 	filter_sizes: [3, 3, 5, 3, 3]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 64


Epoch 1, Training Loss: 2.2282, Training Accuracy: 17.03, 
Validation Loss: 2.1593, Validation Accuracy: 20.91
Epoch 2, Training Loss: 2.1336, Training Accuracy: 22.69, 
Validation Loss: 2.1176, Validation Accuracy: 23.06
Epoch 3, Training Loss: 2.1058, Training Accuracy: 24.41, 
Validation Loss: 2.0977, Validation Accuracy: 23.76
Epoch 4, Training Loss: 2.0597, Training Accuracy: 26.01, 
Validation Loss: 2.0773, Validation Accuracy: 25.06
Epoch 5, Training Loss: 2.0363, Training Accuracy: 27.15, 
Validation Loss: 2.0380, Validation Accuracy: 27.01
Epoch 6, Training Loss: 2.0298, Training Accuracy: 27.32, 
Validation Loss: 2.0541, Validation Accuracy: 25.16
Epoch 7, Training Loss: 2.0173, Training Accuracy: 27.95, 
Validation Loss: 2.0323, Validation Accuracy: 26.96
Epoch 8, Training Loss: 1.9941, Training Accuracy: 28.98, 
Validation Loss: 1.9924, Validation Accuracy: 29.01
Epoch 9, Training Loss: 1.9900, Training Accuracy: 29.65, 
Validation Loss: 1.9759, Validation Accuracy: 29.81
E

...

wandb: Agent Starting Run: tpai7t4d with config:
wandb: 	activation_fn: mish
wandb: 	batch_norm: true
wandb: 	data_augment: true
wandb: 	dropout: 0.3
wandb: 	filter_sizes: [3, 5, 3, 5, 3]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 64


Epoch 1, Training Loss: 2.2195, Training Accuracy: 18.41, 
Validation Loss: 2.1224, Validation Accuracy: 22.86
Epoch 2, Training Loss: 2.1489, Training Accuracy: 22.21, 
Validation Loss: 2.0809, Validation Accuracy: 24.81
Epoch 3, Training Loss: 2.1136, Training Accuracy: 23.56, 
Validation Loss: 2.0945, Validation Accuracy: 24.16
Epoch 4, Training Loss: 2.0997, Training Accuracy: 24.25, 
Validation Loss: 2.0332, Validation Accuracy: 26.61
Epoch 5, Training Loss: 2.0724, Training Accuracy: 25.66, 
Validation Loss: 2.0186, Validation Accuracy: 28.76
Epoch 6, Training Loss: 2.0616, Training Accuracy: 26.26, 
Validation Loss: 2.0098, Validation Accuracy: 29.46
Epoch 7, Training Loss: 2.0632, Training Accuracy: 26.12, 
Validation Loss: 2.0299, Validation Accuracy: 30.32
Epoch 8, Training Loss: 2.0437, Training Accuracy: 27.34, 
Validation Loss: 2.0092, Validation Accuracy: 30.37
Epoch 9, Training Loss: 2.0477, Training Accuracy: 26.41, 
Validation Loss: 1.9582, Validation Accuracy: 30.92
E

train_accuracy,▁▄▅▅▆▇▇▇▇█
training_loss,█▆▄▄▃▃▃▂▂▁
val_accuracy,▁▃▂▄▆▇▇███
val_loss,█▆▇▄▄▃▄▃▁▁
train_accuracy,28.0625
training_loss,2.0168
val_accuracy,30.46523
val_loss,1.96325


wandb: Agent Starting Run: e85o3cmc with config:
wandb: 	activation_fn: mish
wandb: 	batch_norm: true
wandb: 	data_augment: true
wandb: 	dropout: 0.3
wandb: 	filter_sizes: [3, 5, 3, 5, 3]
wandb: 	num_filters: [64, 64, 64, 64, 64]
wandb: 	num_neurons_dense: 128


Epoch 1, Training Loss: 2.2314, Training Accuracy: 17.31, 
Validation Loss: 2.2029, Validation Accuracy: 19.86
